<a href="https://colab.research.google.com/github/Akashpandey1507/Interview-Preparation/blob/main/Pyspark_IPL_2008_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=eadd62fa8dcf9cbb4c5bbc83e337eba72ed3ec7f7dae7d661c7256900527798e
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


In [80]:
file = r"/content/drive/MyDrive/Data Analysis Files/Indian Premier League"

In [3]:
from pyspark.sql import SparkSession
import pyspark

# Create a SparkSession
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .config("spark.some.config.option", "config-value") \
    .getOrCreate()

# Now you can use the 'spark' object to work with Spark APIs.


In [83]:
import os

files = os.listdir(file)
files

['matches.csv', 'deliveries.csv']

In [85]:
match_datasets = spark.read.csv(
    f"{file}/{files[0]}",
    header=True,
                inferSchema=True
                                )
deliveries_datasets = spark.read.csv(
    f"{file}/{files[1]}",
    header=True,
    inferSchema=True
    )

In [86]:
df1 = match_datasets.alias('copy')
df2 = deliveries_datasets.alias('copy')

In [87]:
df1.show()

+---+------+---------+----------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+--------------------+-------------+-------+
| id|season|     city|      date|               team1|               team2|         toss_winner|toss_decision|result|dl_applied|              winner|win_by_runs|win_by_wickets|player_of_match|               venue|             umpire1|      umpire2|umpire3|
+---+------+---------+----------+--------------------+--------------------+--------------------+-------------+------+----------+--------------------+-----------+--------------+---------------+--------------------+--------------------+-------------+-------+
|  1|  2017|Hyderabad|2017-04-05| Sunrisers Hyderabad|Royal Challengers...|Royal Challengers...|        field|normal|         0| Sunrisers Hyderabad|         35|             0|   Yuvraj Singh|Rajiv Gandhi Inte...|         AY Dand

In [88]:
df2.show()

+--------+------+-------------------+--------------------+----+----+------------+------------+-----------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+----------------+--------------+-------------+
|match_id|inning|       batting_team|        bowling_team|over|ball|     batsman| non_striker|     bowler|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|player_dismissed|dismissal_kind|      fielder|
+--------+------+-------------------+--------------------+----+----+------------+------------+-----------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+----------------+--------------+-------------+
|       1|     1|Sunrisers Hyderabad|Royal Challengers...|   1|   1|   DA Warner|    S Dhawan|   TS Mills|            0|        0|       0|          0|          0|           0|           0|         0|         0|            nul

In [90]:

df2 = df2.withColumnRenamed("match_id", "id")

df2.columns

['id',
 'inning',
 'batting_team',
 'bowling_team',
 'over',
 'ball',
 'batsman',
 'non_striker',
 'bowler',
 'is_super_over',
 'wide_runs',
 'bye_runs',
 'legbye_runs',
 'noball_runs',
 'penalty_runs',
 'batsman_runs',
 'extra_runs',
 'total_runs',
 'player_dismissed',
 'dismissal_kind',
 'fielder']

In [91]:
df = df1.join(df2, on= 'id')

In [93]:
df.show()

+---+------+---------+----------+-------------------+--------------------+--------------------+-------------+------+----------+-------------------+-----------+--------------+---------------+--------------------+-----------+--------+-------+------+-------------------+--------------------+----+----+------------+------------+-----------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+----------------+--------------+-------------+
| id|season|     city|      date|              team1|               team2|         toss_winner|toss_decision|result|dl_applied|             winner|win_by_runs|win_by_wickets|player_of_match|               venue|    umpire1| umpire2|umpire3|inning|       batting_team|        bowling_team|over|ball|     batsman| non_striker|     bowler|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|player_dismissed|dismissal_kind|      fielder|
+---+------+---------+--

In [94]:
df1.count()

756

In [95]:
len(df1.columns)

18

In [96]:
df2.count()

179078

In [97]:
len(df2.columns)

21

In [98]:
df.count()

179078

In [99]:
len(df.columns)

38

In [100]:
from pyspark.sql import functions as f



In [102]:
df.select([f.sum(f.col(i).isNull().cast('int')).alias(i) for i in df.columns]).show()

+---+------+----+----+-----+-----+-----------+-------------+------+----------+------+-----------+--------------+---------------+-----+-------+-------+-------+------+------------+------------+----+----+-------+-----------+------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+----------------+--------------+-------+
| id|season|city|date|team1|team2|toss_winner|toss_decision|result|dl_applied|winner|win_by_runs|win_by_wickets|player_of_match|venue|umpire1|umpire2|umpire3|inning|batting_team|bowling_team|over|ball|batsman|non_striker|bowler|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|player_dismissed|dismissal_kind|fielder|
+---+------+----+----+-----+-----+-----------+-------------+------+----------+------+-----------+--------------+---------------+-----+-------+-------+-------+------+------------+------------+----+----+-------+-----------+------+-------------+---------+

In [103]:
[type_col for type_col,type_datatype in df.dtypes if type_datatype in ['int', 'double']]

['id',
 'season',
 'dl_applied',
 'win_by_runs',
 'win_by_wickets',
 'inning',
 'over',
 'ball',
 'is_super_over',
 'wide_runs',
 'bye_runs',
 'legbye_runs',
 'noball_runs',
 'penalty_runs',
 'batsman_runs',
 'extra_runs',
 'total_runs']

In [108]:
intiger_data = df.select([type_col for type_col,type_datatype in df.dtypes if type_datatype in ['int', 'double']])
intiger_data.show()

+---+------+----------+-----------+--------------+------+----+----+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
| id|season|dl_applied|win_by_runs|win_by_wickets|inning|over|ball|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|
+---+------+----------+-----------+--------------+------+----+----+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
|  1|  2017|         0|         35|             0|     1|   1|   1|            0|        0|       0|          0|          0|           0|           0|         0|         0|
|  1|  2017|         0|         35|             0|     1|   1|   2|            0|        0|       0|          0|          0|           0|           0|         0|         0|
|  1|  2017|         0|         35|             0|     1|   1|   3|            0|        0|       0|          0|          0|           

In [109]:
intiger_data.select([f.sum(f.col(i).isNull().cast('int')).alias(i) for i in intiger_data.columns]).show()

+---+------+----------+-----------+--------------+------+----+----+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
| id|season|dl_applied|win_by_runs|win_by_wickets|inning|over|ball|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|
+---+------+----------+-----------+--------------+------+----+----+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
|  0|     0|         0|          0|             0|     0|   0|   0|            0|        0|       0|          0|          0|           0|           0|         0|         0|
+---+------+----------+-----------+--------------+------+----+----+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+



In [105]:
[type_col for type_col,type_datatype in df.dtypes if type_datatype in ['string']]

['city',
 'date',
 'team1',
 'team2',
 'toss_winner',
 'toss_decision',
 'result',
 'winner',
 'player_of_match',
 'venue',
 'umpire1',
 'umpire2',
 'umpire3',
 'batting_team',
 'bowling_team',
 'batsman',
 'non_striker',
 'bowler',
 'player_dismissed',
 'dismissal_kind',
 'fielder']

In [110]:
string_datasets = df.select([type_col for type_col,type_datatype in df.dtypes if type_datatype in ['string']])
string_datasets.show()

+---------+----------+-------------------+--------------------+--------------------+-------------+------+-------------------+---------------+--------------------+-----------+--------+-------+-------------------+--------------------+------------+------------+-----------+----------------+--------------+-------------+
|     city|      date|              team1|               team2|         toss_winner|toss_decision|result|             winner|player_of_match|               venue|    umpire1| umpire2|umpire3|       batting_team|        bowling_team|     batsman| non_striker|     bowler|player_dismissed|dismissal_kind|      fielder|
+---------+----------+-------------------+--------------------+--------------------+-------------+------+-------------------+---------------+--------------------+-----------+--------+-------+-------------------+--------------------+------------+------------+-----------+----------------+--------------+-------------+
|Hyderabad|2017-04-05|Sunrisers Hyderabad|Royal C

In [112]:
string_datasets = string_datasets.select([f.sum(f.col(i).isNull().cast('int')).alias(i) for i in string_datasets.columns])
string_datasets.show()

+----+----+-----+-----+-----------+-------------+------+------+---------------+-----+-------+-------+-------+------------+------------+-------+-----------+------+----------------+--------------+-------+
|city|date|team1|team2|toss_winner|toss_decision|result|winner|player_of_match|venue|umpire1|umpire2|umpire3|batting_team|bowling_team|batsman|non_striker|bowler|player_dismissed|dismissal_kind|fielder|
+----+----+-----+-----+-----------+-------------+------+------+---------------+-----+-------+-------+-------+------------+------------+-------+-----------+------+----------------+--------------+-------+
|1700|   0|    0|    0|          0|            0|     0|   372|            372|    0|    500|    500| 150712|           0|           0|      0|          0|     0|          170244|        170244| 172630|
+----+----+-----+-----+-----------+-------------+------+------+---------------+-----+-------+-------+-------+------------+------------+-------+-----------+------+----------------+---------

In [138]:
df = df.drop(*['umpire3', 'player_dismissed', 'player_dismissed', 'fielder',"dismissal_kind"])

In [139]:
df.select([f.sum(f.col(i).isNull().cast('int')).alias(i) for i in df.columns]).show()

+---+------+----+----+-----+-----+-----------+-------------+------+----------+------+-----------+--------------+---------------+-----+-------+-------+------+------------+------------+----+----+-------+-----------+------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
| id|season|city|date|team1|team2|toss_winner|toss_decision|result|dl_applied|winner|win_by_runs|win_by_wickets|player_of_match|venue|umpire1|umpire2|inning|batting_team|bowling_team|over|ball|batsman|non_striker|bowler|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|
+---+------+----+----+-----+-----+-----------+-------------+------+----------+------+-----------+--------------+---------------+-----+-------+-------+------+------------+------------+----+----+-------+-----------+------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
|  0|     0|1700|   0|

In [140]:
string_datasets = df.select([type_col for type_col,type_datatype in df.dtypes if type_datatype in ['string']])
string_datasets.show()

+---------+----------+-------------------+--------------------+--------------------+-------------+------+-------------------+---------------+--------------------+-----------+--------+-------------------+--------------------+------------+------------+-----------+
|     city|      date|              team1|               team2|         toss_winner|toss_decision|result|             winner|player_of_match|               venue|    umpire1| umpire2|       batting_team|        bowling_team|     batsman| non_striker|     bowler|
+---------+----------+-------------------+--------------------+--------------------+-------------+------+-------------------+---------------+--------------------+-----------+--------+-------------------+--------------------+------------+------------+-----------+
|Hyderabad|2017-04-05|Sunrisers Hyderabad|Royal Challengers...|Royal Challengers...|        field|normal|Sunrisers Hyderabad|   Yuvraj Singh|Rajiv Gandhi Inte...|AY Dandekar|NJ Llong|Sunrisers Hyderabad|Royal Ch

In [141]:
string_datasets.select([f.sum(f.col(i).isNull().cast('int')).alias(i) for i in string_datasets.columns]).show()

+----+----+-----+-----+-----------+-------------+------+------+---------------+-----+-------+-------+------------+------------+-------+-----------+------+
|city|date|team1|team2|toss_winner|toss_decision|result|winner|player_of_match|venue|umpire1|umpire2|batting_team|bowling_team|batsman|non_striker|bowler|
+----+----+-----+-----+-----------+-------------+------+------+---------------+-----+-------+-------+------------+------------+-------+-----------+------+
|1700|   0|    0|    0|          0|            0|     0|   372|            372|    0|    500|    500|           0|           0|      0|          0|     0|
+----+----+-----+-----+-----------+-------------+------+------+---------------+-----+-------+-------+------------+------------+-------+-----------+------+



In [142]:
df = df.fillna('Unknown')
df.show()

+---+------+---------+----------+-------------------+--------------------+--------------------+-------------+------+----------+-------------------+-----------+--------------+---------------+--------------------+-----------+--------+------+-------------------+--------------------+----+----+------------+------------+-----------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
| id|season|     city|      date|              team1|               team2|         toss_winner|toss_decision|result|dl_applied|             winner|win_by_runs|win_by_wickets|player_of_match|               venue|    umpire1| umpire2|inning|       batting_team|        bowling_team|over|ball|     batsman| non_striker|     bowler|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|
+---+------+---------+----------+-------------------+--------------------+--------------------+-------------+------+----------+-----

In [143]:
df.select([f.sum(f.col(i).isNull().cast('int')).alias(i) for i in df.columns]).show()

+---+------+----+----+-----+-----+-----------+-------------+------+----------+------+-----------+--------------+---------------+-----+-------+-------+------+------------+------------+----+----+-------+-----------+------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
| id|season|city|date|team1|team2|toss_winner|toss_decision|result|dl_applied|winner|win_by_runs|win_by_wickets|player_of_match|venue|umpire1|umpire2|inning|batting_team|bowling_team|over|ball|batsman|non_striker|bowler|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|
+---+------+----+----+-----+-----+-----------+-------------+------+----------+------+-----------+--------------+---------------+-----+-------+-------+------+------------+------------+----+----+-------+-----------+------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
|  0|     0|   0|   0|

In [144]:
df.count()

179078

In [145]:
len(df.columns)

34

In [146]:
df.columns

['id',
 'season',
 'city',
 'date',
 'team1',
 'team2',
 'toss_winner',
 'toss_decision',
 'result',
 'dl_applied',
 'winner',
 'win_by_runs',
 'win_by_wickets',
 'player_of_match',
 'venue',
 'umpire1',
 'umpire2',
 'inning',
 'batting_team',
 'bowling_team',
 'over',
 'ball',
 'batsman',
 'non_striker',
 'bowler',
 'is_super_over',
 'wide_runs',
 'bye_runs',
 'legbye_runs',
 'noball_runs',
 'penalty_runs',
 'batsman_runs',
 'extra_runs',
 'total_runs']

In [153]:
df.dtypes

[('id', 'int'),
 ('season', 'int'),
 ('city', 'string'),
 ('date', 'string'),
 ('team1', 'string'),
 ('team2', 'string'),
 ('toss_winner', 'string'),
 ('toss_decision', 'string'),
 ('result', 'string'),
 ('dl_applied', 'int'),
 ('winner', 'string'),
 ('win_by_runs', 'int'),
 ('win_by_wickets', 'int'),
 ('player_of_match', 'string'),
 ('venue', 'string'),
 ('umpire1', 'string'),
 ('umpire2', 'string'),
 ('inning', 'int'),
 ('batting_team', 'string'),
 ('bowling_team', 'string'),
 ('over', 'int'),
 ('ball', 'int'),
 ('batsman', 'string'),
 ('non_striker', 'string'),
 ('bowler', 'string'),
 ('is_super_over', 'int'),
 ('wide_runs', 'int'),
 ('bye_runs', 'int'),
 ('legbye_runs', 'int'),
 ('noball_runs', 'int'),
 ('penalty_runs', 'int'),
 ('batsman_runs', 'int'),
 ('extra_runs', 'int'),
 ('total_runs', 'int')]

In [147]:
#string columns

[string_data for string_data, string_type in df.dtypes if string_type in ['string']]


['city',
 'date',
 'team1',
 'team2',
 'toss_winner',
 'toss_decision',
 'result',
 'winner',
 'player_of_match',
 'venue',
 'umpire1',
 'umpire2',
 'batting_team',
 'bowling_team',
 'batsman',
 'non_striker',
 'bowler']

In [148]:
df.select(
[string_data for string_data, string_type in df.dtypes if string_type in ['string']]).show()

+---------+----------+-------------------+--------------------+--------------------+-------------+------+-------------------+---------------+--------------------+-----------+--------+-------------------+--------------------+------------+------------+-----------+
|     city|      date|              team1|               team2|         toss_winner|toss_decision|result|             winner|player_of_match|               venue|    umpire1| umpire2|       batting_team|        bowling_team|     batsman| non_striker|     bowler|
+---------+----------+-------------------+--------------------+--------------------+-------------+------+-------------------+---------------+--------------------+-----------+--------+-------------------+--------------------+------------+------------+-----------+
|Hyderabad|2017-04-05|Sunrisers Hyderabad|Royal Challengers...|Royal Challengers...|        field|normal|Sunrisers Hyderabad|   Yuvraj Singh|Rajiv Gandhi Inte...|AY Dandekar|NJ Llong|Sunrisers Hyderabad|Royal Ch

In [151]:
# to show only numerical columns

[num_data for num_data, num_type in df.dtypes if num_type in ['int', 'double']]

['id',
 'season',
 'dl_applied',
 'win_by_runs',
 'win_by_wickets',
 'inning',
 'over',
 'ball',
 'is_super_over',
 'wide_runs',
 'bye_runs',
 'legbye_runs',
 'noball_runs',
 'penalty_runs',
 'batsman_runs',
 'extra_runs',
 'total_runs']

In [152]:
df.select([num_data for num_data, num_type in df.dtypes if num_type in ['int', 'double']]).show()

+---+------+----------+-----------+--------------+------+----+----+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
| id|season|dl_applied|win_by_runs|win_by_wickets|inning|over|ball|is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|
+---+------+----------+-----------+--------------+------+----+----+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
|  1|  2017|         0|         35|             0|     1|   1|   1|            0|        0|       0|          0|          0|           0|           0|         0|         0|
|  1|  2017|         0|         35|             0|     1|   1|   2|            0|        0|       0|          0|          0|           0|           0|         0|         0|
|  1|  2017|         0|         35|             0|     1|   1|   3|            0|        0|       0|          0|          0|           

In [154]:
df.show(truncate=False)

+---+------+---------+----------+-------------------+---------------------------+---------------------------+-------------+------+----------+-------------------+-----------+--------------+---------------+-----------------------------------------+-----------+--------+------+-------------------+---------------------------+----+----+------------+------------+-----------+-------------+---------+--------+-----------+-----------+------------+------------+----------+----------+
|id |season|city     |date      |team1              |team2                      |toss_winner                |toss_decision|result|dl_applied|winner             |win_by_runs|win_by_wickets|player_of_match|venue                                    |umpire1    |umpire2 |inning|batting_team       |bowling_team               |over|ball|batsman     |non_striker |bowler     |is_super_over|wide_runs|bye_runs|legbye_runs|noball_runs|penalty_runs|batsman_runs|extra_runs|total_runs|
+---+------+---------+----------+---------------